In [1]:
#Produces some data that allows you to test the interactive chart without using data
from faker import Faker
fake = Faker()

In [2]:
import json
with open('random_images.json') as data_file:    
    random_images = json.load(data_file)

In [3]:
import requests
import json
import random

def create_node():
    node = {}
    node["full_name"] = fake.name()
    node["email_address"] = fake.email()
    node["phone_numbers"] = fake.phone_number()
    node["id"] = fake.md5()
    node["summary"] = fake.text()
    node["job_title"] = fake.job()
    node["web_url"] = fake.url()
    
    node["colour"] = fake.safe_hex_color()
    
    node["statistic_1"] = fake.random_number(3)
    node["statistic_2"] = fake.random_number(1)
    
    node["mugshot_url_template"] = "http://lorempixel.com/200/200"
    #If you have access to a list of images you can use this:
    node["mugshot_url_template"] = random.choice(random_images)["image_urls"]["epic"]
    
    

    return node
root = create_node()

In [4]:
root = create_node()
import numpy as np
import random

from scipy import stats

def create_random_children(node,depth):
    
    num_children = np.random.binomial(8-depth, 0.6, 1)[0]
    
    node["children"] = []
    
    for i in range(num_children):
        this_node = create_node()
        node["children"].append(this_node)
    
    for node in node["children"]:
        create_random_children(node, depth+1)
        
create_random_children(root,1)

In [5]:

#ALso need data that for the select box
select_box = []


def add_to_select_box(node):

    new_item = {}
    new_item["id"] = node["id"]
    new_item["text"] =  node["full_name"] + " " +node["job_title"]
    select_box.append(new_item)
    
    if "children" in node:
        for c in node["children"]:
            add_to_select_box(c)


#Iterate through the nodes, adding to select box

add_to_select_box(root)    
select_box.sort(key=lambda x: x["text"])

In [6]:
final_data = {"tree": root, "select_box":select_box}

In [7]:
flat = []

def flatten(tree):
    """
    Recursively iterate through tree adding rows to the flat array
    """

    def next_node(node, parent_id):
        
        this_row = {}
        
        this_row["parent"] = parent_id
        
        this_row["full_name"] = node["full_name"]
        this_row["email_address"] = node["email_address"]
        this_row["phone_numbers"] = node["phone_numbers"]
        this_row["id"] = node["id"]
        this_row["summary"] = node["summary"]
        this_row["job_title"] = node["job_title"]
        this_row["web_url"] = node["web_url"]
        this_row["colour"] = node["colour"]
        this_row["mugshot_url_template"] = node["mugshot_url_template"]
        
        this_row["statistic_1"] = node["statistic_1"] 
        this_row["statistic_2"] = node["statistic_2"] 
        
        flat.append(this_row)
        
        for child in node["children"]:
            next_node(child, node["id"])
    
    next_node(tree, "")
    
flatten(root)
        
import pandas as pd
df = pd.DataFrame(flat)
df.to_csv("faked_data_flat_file.csv", encoding="utf-8", index=False)